# Module 5.2 - Abrupt Regime Switch

## Objective

Examine how rolling performance metrics respond to a sudden structural break in edge.

Unlike gradual drift (Submodule 5.1), this experiment introduces an abrupt regime change.

## Simulation Design

- Trades 1–500: 55% win rate (positive edge)
- Trades 501–1000: 45% win rate (negative edge)
- Sudden switch at trade 500

This creates a clear structural break.     

## Tasks

1. Use rolling win-rate and rolling expectancy to detect the change.
2. Measure detection delay after the switch.
3. Count false alarms (detections before trade 500).
4. Run multiple Monte Carlo simulations to measure variability.

## Core Questions

- How delayed is detection after a real regime change?
- How often does variance falsely mimic a regime switch?
- What distinguishes real structural breakdown from noise?

## Hypothesis

A true regime break should produce persistent deterioration in rolling metrics.     
However, detection will lag due to window averaging.     
Shorter windows may detect earlier but increase false alarms.     

This module tests the trade-off between responsiveness and reliability

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view

## Global variables

In [2]:
n_trades = 1000
window = 100     # Try different window sizes
win = 1
loss = -1
n_runs = 50

## Simulation Function

In [3]:
def simulate_regime_switch(n_trades = n_trades, window = window):
    
    # ----True regime----
    p = np.zeros(n_trades)
    p[:500] = 0.55     # positive edge
    p[500:] = 0.44     # negative edge

    # --- generate trades -------
    seq = np.where(
        np.random.rand(n_trades) < p,
        win, loss
    )
    equity = np.cumsum(seq)

    # --- rolling matrics -----
    wins = (seq == 1).astype(int)

    rolling_win = sliding_window_view(wins, window).mean(axis = 1)

    # for a 1R system Expectancy = 2p - 1
    
    rolling_exp = 2*rolling_win -1

    return {
        "equity": equity,
        "rolling_win": rolling_win,
        "rolling_exp": rolling_exp,
        "true_p": p
    }



## Detection Logic

In [4]:
def detect_change(rolling_series, threshold = 0.50, window = window, persistence = 20):
    below = np.where(rolling_series < threshold)[0]

    if len(below) == 0:
        return None

    # Convert window index to trade index
    return below[0] + window - 1


## Monte Carlo

In [5]:
detections = []

for _ in range(n_runs):
    sim = simulate_regime_switch()
    detect_trade = detect_change(sim['rolling_win'])
    detections.append(detect_trade)


In [6]:
switch_point = 500

valid_detections = [d for d in detections if d is not None]

false_alarms = [d for d in valid_detections if d < switch_point]

true_detections = [d for d in valid_detections if d >= switch_point]

print("Mean detection trade:", round(np.mean(valid_detections)))
print("False alarms:", len(false_alarms))
print("Never detected edge shift:", detections.count(None))
print('correct Regime shift detection: ' , len(true_detections))

if len(true_detections) > 0:
    print(f"Mean detection delay after switch: ~{np.mean(np.array(true_detections) - switch_point):.0f} trades after the Regime shift")


print(f"\n\n{(len(false_alarms) / len(valid_detections))*100}% of all detections were False")

Mean detection trade: 258
False alarms: 45
Never detected edge shift: 0
correct Regime shift detection:  5
Mean detection delay after switch: ~49 trades after the Regime shift


90.0% of all detections were False


# Observation
- A simple rolling threshold (<50%) produces a very high false alarm rate when true edge is only moderately above 50%.
- Even a clear structural break (55% → 44%) is difficult to detect cleanly in real time.
- Detection is inherently delayed due to rolling window smoothing.
- Sampling variance frequently mimics structural failure.
- Naive threshold-based detection is unreliable for edge monitoring.
- True edge is unobservable.
- Regime detection is probabilistic, not certain.

Core Insight:

Online regime detection is fundamentally a signal-to-noise problem.    
When edge is small, noise dominates detection behavior.